"""VLP_classification.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/19yn-ef-FixANlQFJTMEwD9GPb_9jkKPt
"""

In [ ]:
import os
os.chdir("drive/MyDrive/VLP_classify")

In [ ]:
!pwd

In [ ]:
import pandas as pd
import numpy as np
data =  pd.read_csv("testdata.csv", header=None)
statis = []
for index, row in data.iterrows():
  count=0
  for col in row:
    if not np.isnan(col):
      count+=1
  statis.append((index,count))
print(statis)
sorted_list = sorted(statis, key=lambda t: t[1])
print(sorted_list) # sort data by row length

In [ ]:
len(sorted_list)

In [ ]:
sorted_list[200]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
bar = [t[1] for t in sorted_list]
plt.bar(range(len(bar)), bar)

In [ ]:
index = [t[0] for t in sorted_list]
print(index)

In [ ]:
temp = data.iloc[535]
temp = list(i for i in temp if not np.isnan(i))
print(len(temp))

In [ ]:
resample_data = []
label = []
for i in index:
  temp = []
  for ind, val in enumerate(data.iloc[i]):
    if ind == 0:
      label.append(val)
    elif not np.isnan(val):
      temp.append(val)
  resample_data.append(pd.Series(temp))
print(len(resample_data))
print(len(label))

In [ ]:
resample_data[0]

In [ ]:
test = resample_data[0]
new_index = pd.Index(np.linspace(0, len(test) - 1, num=1000))
#print(new_index)
interpolated_sequence = pd.Series(np.interp(new_index, test.index, test.values), index=new_index)

# The sequence after supplementing to the target length
print(interpolated_sequence.values)

In [ ]:
cleaned_data = []
target_length = 500
for seq in resample_data:
  if len(seq)<=target_length:
    # make up length
    new_index = pd.Index(np.linspace(0, len(seq)-1, num=target_length))
    new_seq = np.interp(new_index, seq.index, seq.values)
    cleaned_data.append(new_seq)
  else:
    num = len(seq)
    new_index = pd.Index(np.linspace(0, len(seq)-1, num=target_length))
    new_seq = np.interp(new_index, seq.index, seq.values)
    cleaned_data.append(new_seq)

In [ ]:
import matplotlib.pyplot as plt
bar = [len(t) for t in cleaned_data]
plt.bar(range(len(bar)), bar)

In [ ]:
for index, values in enumerate(cleaned_data):
  plt.plot(range(500), values)
plt.show()

In [ ]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=5).fit_transform(np.asarray(cleaned_data))
X_embedded.shape

In [ ]:
fig = plt.figure(figsize=(8,5))
for i in range(2):
  print(i)
  x = []
  print(type(x))
  y = []
  for index, j in enumerate(label):
    if j == i:
      x.append(X_embedded[index][0])
      y.append(X_embedded[index][1])
  plt.scatter(x, y)

plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers

In [ ]:
# prepare train data and label
data_sequences = np.array(cleaned_data)
label_encoder = LabelEncoder()
data_labels = label_encoder.fit_transform(label)  # labels for training data
# print(data_labels)

# # calculate 1-Norm bumber for each line
# norms = np.linalg.norm(data_sequences, axis=1, keepdims=True)
# # standardization for each line
# data_sequences = data_sequences / norms

# seperate the train and test dataset
X_train, X_test, y_train, y_test = train_test_split(data_sequences, data_labels, test_size=0.2, random_state = 7)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.keras", save_best_only=True, monitor="binary_accuracy", mode='max'
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="binary_accuracy", mode='max', patience=50, verbose=1),
]

In [ ]:
print(len([x for x in y_train if x==0])/ len(y_train))

In [ ]:
# CNN
regularizer = 0
input = keras.layers.Input(shape=(X_train.shape[1],1))
# x = keras.layers.Reshape((X_train.shape[1], 1))(input)
x = keras.layers.Conv1D(32, 10, activation='relu',kernel_regularizer=keras.regularizers.l2(regularizer))(input)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)

x = keras.layers.Conv1D(64, 20, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)

x = keras.layers.Conv1D(64, 10, activation='relu',kernel_regularizer=keras.regularizers.l2(regularizer))(x)
x = keras.layers.GlobalAvgPool1D()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(10, activation='relu')(x)
output = keras.layers.Dense(1, activation='sigmoid')(x)

mymodel = keras.Model(input, output)
mymodel.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=[keras.metrics.BinaryAccuracy()]
)
mymodel.summary()

mymodel.fit(X_train,
            y_train,
            epochs=500,
            batch_size=4,
            validation_data=(X_test, y_test),
            callbacks=callbacks,
            verbose=1,
            )

model = keras.models.load_model("best_model.keras")
model.evaluate(X_test,y_test)

In [ ]:
import keras
mymodel = keras.models.load_model("best_model_0910.keras")

In [ ]:
def create_truncated_model(trained_model):
  regularizer = 0
  input = keras.layers.Input(shape=(X_train.shape[1],1))
  x = keras.layers.Conv1D(32, 10, activation='relu',kernel_regularizer=keras.regularizers.l2(regularizer))(input)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  x = keras.layers.Conv1D(64, 20, activation='relu')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.ReLU()(x)

  x = keras.layers.Conv1D(64, 10, activation='relu',kernel_regularizer=keras.regularizers.l2(regularizer))(x)
  x = keras.layers.GlobalAvgPool1D()(x)
  x = keras.layers.Flatten()(x)
  output = keras.layers.Dense(10, activation='relu')(x)
  new_model = keras.Model(input, output)

  for i, layer in enumerate(new_model.layers):
    layer.set_weights(trained_model.layers[i].get_weights())

  new_model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.RMSprop(learning_rate=0.0005),
    metrics=[keras.metrics.BinaryAccuracy()]
  )

  return new_model

truncated_model = create_truncated_model(mymodel)
hidden_features = truncated_model.predict(X_train)

Q393L+WT : 0;blue
L53D+V108D : 1;orange

In [ ]:
X_embedded = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=50).fit_transform(np.asarray(hidden_features))
fig = plt.figure(figsize=(10,10),dpi=300)
for i in range(2):
  indices = np.where(y_train==i)
  plt.scatter(X_embedded[indices, 0], X_embedded[indices, 1], label=str(i))
#plt.legend()
plt.gca().spines[['top','bottom','left','right']].set_linewidth(3)
plt.xticks([])
plt.yticks([])
# plt.xlabel("Dimension 1", size=10)
# plt.ylabel("Dimension 2", size=10)
plt.legend(['WT, Q393L','L53D, V108D'], fontsize="30", markerscale=3)
plt.show()